In [ ]:
import torch

state_dict = torch.load("saved_models/model_state_dict.pth", map_location="cpu")
state_dict

In [ ]:
from gis.dino import Dinov2ForSemanticSegmentation

id2label = {
	0: "baclgrpimd",
	1: "track",
}

model = Dinov2ForSemanticSegmentation.from_pretrained(
    "facebook/dinov2-base", id2label=id2label, num_labels=len(id2label)
)
model.load_state_dict(state_dict)

In [ ]:
from gis.config import Config

config = Config()

import os 


labels = [x for x in os.listdir(config.mnt_path / 'label/18')]
labels

coords = []
for label in labels:
    x,y = label.split('_')
    x, y = int(x), int(y.replace('.npy', ''))
    coords.append((18, x, y))
coords

import cv2 
import matplotlib.pyplot as plt 
import numpy as np


def load_image(coord):
    z,x,y = coord
    filepath = config.mnt_path / f'image/18/{z}_{x}_{y}.jpg'
    img = cv2.imread(filepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img 

def load_mask(coord):
    z,x,y = coord
    filepath = config.mnt_path / f'label/18/{x}_{y}.npy'
    mask = np.load(filepath)
    return mask


images = []
for coord in coords:
    img = load_image(coord)
    images.append(img)
    #print(img)
    break
    mask = load_mask(coord)
    fig, axs = plt.subplots(1,2,figsize=(10, 5))
    axs[0].imshow(img)
    axs[1].imshow(mask)
    plt.show()
image = images[0]
plt.imshow(image)

In [ ]:
import albumentations as A

ADE_MEAN = np.array([123.675, 116.280, 103.530]) / 255
ADE_STD = np.array([58.395, 57.120, 57.375]) / 255
width = 256


val_transform = A.Compose(
    [
        A.Resize(width=width, height=width),
        A.Normalize(mean=ADE_MEAN, std=ADE_STD),
    ]
)
# 
image = val_transform(image = image)['image']

In [ ]:
#.unsqueeze(1)
i = torch.from_numpy(image[np.newaxis, ...]).permute(0, 3, 1, 2)
print(i.shape)

out = model(i)[0].detach()
print(out.shape)
plt.imshow(out[0] > 0)

In [ ]:
out[0]